# Self-ask with search

This walkthrough showcases the self-ask with search chain.

In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.llms import Fireworks
from langchain_community.tools.tavily_search import TavilyAnswer

## Initialize the tools

We will initialize the tools we want to use. This is a good tool because it gives us **answers** (not documents)

For this agent, only one tool can be used and it needs to be named "Intermediate Answer"

In [2]:
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]

## Create the Agent

In [3]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")

In [4]:
print(prompt)

input_variables=['agent_scratchpad', 'input'] template='Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington 

In [5]:
# Choose the LLM that will drive the agent
llm = Fireworks()

# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

## Run the agent

In [6]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?The reigning men's U.S. Open champion is Novak Djokovic. He won his 24th Grand Slam singles title at the 2023 US Open by defeating Daniil Medvedev in the final with a score of 6-3, 7-6 (5), 6-3. Djokovic's victory tied Margaret Court's record and solidified his case as one of the greatest tennis players of all time.
So the final answer is: Belgrade, Serbia

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Belgrade, Serbia'}

## Using LangChain Expression Language

First we will show how to construct this agent from components using LangChain Expression Language

In [8]:
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import SelfAskOutputParser

In [9]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [10]:
llm_with_stop = llm.bind(stop=["\nIntermediate answer:"])

In [11]:
agent = (
    {
        "input": lambda x: x["input"],
        # Use some custom observation_prefix/llm_prefix for formatting
        "agent_scratchpad": lambda x: format_log_to_str(
            x["intermediate_steps"],
            observation_prefix="\nIntermediate answer: ",
            llm_prefix="",
        ),
    }
    | prompt
    | llm_with_stop
    | SelfAskOutputParser()
)

In [12]:
from langchain.agents import AgentExecutor

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke(
    {"input": "What is the hometown of the reigning men's U.S. Open champion?"}
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: What is the current men's U.S. Open champion?The current men's U.S. Open champion is Novak Djokovic. He won his fourth title at the US Open and 24th Grand Slam title by defeating Daniil Medvedev in the final.
So the final answer is: Novak Djokovic

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Novak Djokovic'}